# Accessing Object Table data using the Butler

## Import necessary modules

In [1]:
from lsst.daf.butler import Butler
from collections import defaultdict
import numpy as np
from astropy import units as u

## Set up the butler

In [2]:
instrument = 'LSSTComCam'
collections = ['LSSTComCam/DP1/defaults', 
               'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260',
               'skymaps', ]
skymap = 'lsst_cells_v1'
butler = Butler("/repo/dp1",
                instrument=instrument, 
                collections=collections, 
                skymap=skymap)
registry = butler.registry
skymap = butler.get('skyMap', skymap=skymap)

## Which tracts cover each field?

In [3]:
fields = defaultdict(set)
with butler.query() as base_query:
    processed_visit_query = base_query.join_dataset_search("visit_summary").where('visit > 0')
    for row in processed_visit_query.general(["tract", "band"], "visit.target_name"):
        if row["visit.target_name"] != 'slew_icrs':
            fields[row["visit.target_name"]].add(row["tract"])

## Estimage the median 5\sigma depth of each field

In [4]:
fieldOrder = ['47_Tuc', 'ECDFS', 'EDFS_comcam', 'Fornax_dSph', 'Rubin_SV_095_-25', 'Rubin_SV_38_7', 'Seagull']
bands = ['u','g','r','i','z','y']

with open("dp1_depths.tex", "w") as f:
    f.write(r"""%%%%% This table is auto generated from data, DO NOT EDIT
\setlength{\tabcolsep}{6pt}  % default is 6pt
\begin{deluxetable}{lcccccc}
\tablecaption{Median $5\sigma$ coadd detection limits per field and band..
\label{tab:dp1_m5_depths} }

\tablehead{
  \textbf{Field Code} & \multicolumn{6}{c}{\textbf{Band}}\\
  \cline{2-7}
   &u&g&r&i&z&y
}
\startdata
""")

    for fieldName in fieldOrder:
        latexName = fieldName.replace('_', '\\_')
        medmags = ''
        for band in bands:
            mags = np.array([])
            tracts = fields[fieldName]
            for tract in tracts:
                columns = [f'{band}_psfFlux',
                       f'{band}_psfFluxErr',
                       f'{band}_psfFlux_flag',
                       f'{band}_extendedness']
                table = butler.get('object',
                                   tract=tract,
                                   skymap='lsst_cells_v1',
                                   parameters={'columns':columns})
            
                sn = table[f'{band}_psfFlux'] / table[f'{band}_psfFluxErr']
                if len(table) > 0:
                    starSelection = (
                        (~table[f'{band}_psfFlux_flag']) &
                        (table[f'{band}_extendedness'] <= 0.5) &
                        ~table[f'{band}_extendedness'].mask &
                        (sn > 4.9) &
                        (sn < 5.1))
                    mags = np.append(mags,(table[starSelection][f'{band}_psfFlux'] * u.nJy).to_value(u.ABmag))
            medmagstr = f'{np.median(mags):.2f}'
            if 'nan' in medmagstr:
                medmagstr = '-'
            medmags = medmags + '&' + medmagstr
        f.write(f'{latexName}{medmags}\\\\\n')
    f.write(r"""\enddata
\end{deluxetable}
""")
f.close()

/opt/lsst/software/stack/conda/envs/lsst-scipipe-10.0.0/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/lsst/software/stack/conda/envs/lsst-scipipe-10.0.0/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
